# Pdt_Clf - Logistic Regression

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import joblib

# Load Data

In [2]:
df = pd.read_parquet(r"input_data\ccd_labeled_train_validate.parquet")

print(df.shape)
display(df.head())                    

(13268, 6)


date              dlg_id  \
0  2024-10-05  2024-10-07_1017--0   
1  2024-10-05  2024-10-07_1017--1   
2  2024-10-05  2024-10-07_1017--2   
3  2024-10-05  2024-10-07_1017--3   
4  2024-10-05  2024-10-07_1017--4   

                                                text product   action  \
0                      i am still waiting on my card    card  inquire   
1  what can i do if my card still hasnt arrived a...    card  inquire   
2  i have been waiting over a week is the card st...    card  inquire   
3  can i track my card while it is in the process...    card  inquire   
4  how do i know if i will get my card or if it i...    card  inquire   

  sentiment  
0  negative  
1  negative  
2  negative  
3   neutral  
4   neutral

In [3]:
# testing - check for duplicates or errors in labels
sorted(df['product'].unique())

['account',
 'atm',
 'card',
 'credit_card',
 'current_account',
 'debit_card',
 'fixed_deposit',
 'forex_account',
 'loan',
 'mobile_app',
 'others',
 'savings_account',
 'website']

In [4]:
# df['product'].value_counts()

In [5]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['product'].to_list(), test_size=0.2, random_state=42, stratify=df['product'].to_list())

In [6]:
# Preprocess the text data using TF-IDF
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [7]:
# Train a classifier (Logistic Regression)
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

LogisticRegression()

# Predict X_test

# Error Duplication predicted Labels

In [8]:
# SMake predictions on the test set
y_pred = model.predict(X_test_tfidf)

# Evaluate the model
print("Accuracy:", round(accuracy_score(y_test, y_pred), 3))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.767

Classification Report:
                  precision    recall  f1-score   support

        account       0.73      0.86      0.79       416
            atm       0.84      0.79      0.81       212
           card       0.85      0.93      0.89       730
    credit_card       0.65      0.50      0.57       111
current_account       0.70      0.58      0.64       151
     debit_card       0.56      0.53      0.54       114
  fixed_deposit       0.77      0.62      0.68        81
  forex_account       0.78      0.79      0.79       260
           loan       0.81      0.66      0.73        83
     mobile_app       0.83      0.82      0.82       163
         others       0.49      0.31      0.38        80
savings_account       0.63      0.59      0.61       164
        website       0.71      0.74      0.73        89

       accuracy                           0.77      2654
      macro avg       0.72      0.67      0.69      2654
   weighted avg       0.76      0.77      0.7

# Predict a single text

In [9]:
# To make a prediction on new text
def predict_category(text):
    text_tfidf = vectorizer.transform([text])
    prediction = model.predict(text_tfidf)
    return prediction[0]

# Example usage
example_text = "my card is stuck in the atm machine. I want it back."
print("Predicted category:", predict_category(example_text))

Predicted category: atm


# Save the Model

In [10]:
# Save the model using joblib
joblib.dump(model, 'trained_model/pdt_clr_logistic_regression_model.pkl')

print("Model saved!")

Model saved!


# Reload Model and do Predict

In [11]:
del model

In [12]:
def predict_category(text):
    text_tfidf = vectorizer.transform([text])
    prediction = model.predict(text_tfidf)
    return prediction[0]

model = joblib.load('trained_model/pdt_clr_logistic_regression_model.pkl')

# Example usage
example_text = "Why are your fixed deposit rates so terrible? I don't want to roll over my fixed deposit."
print("Predicted category:", predict_category(example_text))

Predicted category: fixed_deposit


# Predict X Test

In [13]:
# Load the model from the file
loaded_model = joblib.load('trained_model/pdt_clr_logistic_regression_model.pkl')

# Make predictions using the loaded model
predictions = loaded_model.predict(X_test_tfidf)

# # Print the predictions
# print("Predictions:", predictions)

# Evaluate the model
print("Accuracy:", round(accuracy_score(y_test, y_pred),3))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.767

Classification Report:
                  precision    recall  f1-score   support

        account       0.73      0.86      0.79       416
            atm       0.84      0.79      0.81       212
           card       0.85      0.93      0.89       730
    credit_card       0.65      0.50      0.57       111
current_account       0.70      0.58      0.64       151
     debit_card       0.56      0.53      0.54       114
  fixed_deposit       0.77      0.62      0.68        81
  forex_account       0.78      0.79      0.79       260
           loan       0.81      0.66      0.73        83
     mobile_app       0.83      0.82      0.82       163
         others       0.49      0.31      0.38        80
savings_account       0.63      0.59      0.61       164
        website       0.71      0.74      0.73        89

       accuracy                           0.77      2654
      macro avg       0.72      0.67      0.69      2654
   weighted avg       0.76      0.77      0.7